## Installing Dependencies

In [1]:
pip install ultralytics==8.0.196

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 17.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 33.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.10.0.82
    Uninstalling opencv-python-headless-4.10.0.82:
      Successfully uninstalled opencv-python-headless-4.10.0.82
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you 

In [3]:
pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 52.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


## Importing Libraries

In [4]:
import ultralytics
import torch
from ultralytics import YOLO
import yaml
import os
import cv2
import time
import numpy as np
import av


## Loading the Dataset

In [5]:


from roboflow import Roboflow
rf = Roboflow(api_key="MaXSsjPsJ1tghm68R4jI")
project = rf.workspace("boycott-brands").project("cola-vs-pepsi")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cola-vs-pepsi--1 in yolov8:: 100%|██████████| 544/544 [00:00<00:00, 6683.48it/s]


The dataset is in Yolov8 format

In [6]:
# Reading the yaml.data

def read_data_yaml(yaml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    
    return data

In [7]:

yaml_path = "/kaggle/working/cola-vs-pepsi--1/data.yaml"
data = read_data_yaml(yaml_path)
    
print(data)

{'names': ['Coca-Cola', 'Pepsi'], 'nc': 2, 'roboflow': {'license': 'CC BY 4.0', 'project': 'cola-vs-pepsi', 'url': 'https://universe.roboflow.com/boycott-brands/cola-vs-pepsi/dataset/1', 'version': 1, 'workspace': 'boycott-brands'}, 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}


In [8]:
data

{'names': ['Coca-Cola', 'Pepsi'],
 'nc': 2,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'cola-vs-pepsi',
  'url': 'https://universe.roboflow.com/boycott-brands/cola-vs-pepsi/dataset/1',
  'version': 1,
  'workspace': 'boycott-brands'},
 'test': 'test/images',
 'train': 'train/images',
 'val': 'valid/images'}

In [9]:
data['train'] = '/kaggle/working/cola-vs-pepsi--1/train/images'
data['test'] = '/kaggle/working/cola-vs-pepsi--1/test/images'
data['valid'] = '/kaggle/working/cola-vs-pepsi--1/valid/images'

In [10]:
# if 'val' in data:
#     del data['val']

In [11]:
data

{'names': ['Coca-Cola', 'Pepsi'],
 'nc': 2,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'cola-vs-pepsi',
  'url': 'https://universe.roboflow.com/boycott-brands/cola-vs-pepsi/dataset/1',
  'version': 1,
  'workspace': 'boycott-brands'},
 'test': '/kaggle/working/cola-vs-pepsi--1/test/images',
 'train': '/kaggle/working/cola-vs-pepsi--1/train/images',
 'val': 'valid/images',
 'valid': '/kaggle/working/cola-vs-pepsi--1/valid/images'}

In [12]:
with open(yaml_path, 'w') as file:
    yaml.safe_dump(data, file)

In [13]:
data = read_data_yaml(yaml_path)
data

{'names': ['Coca-Cola', 'Pepsi'],
 'nc': 2,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'cola-vs-pepsi',
  'url': 'https://universe.roboflow.com/boycott-brands/cola-vs-pepsi/dataset/1',
  'version': 1,
  'workspace': 'boycott-brands'},
 'test': '/kaggle/working/cola-vs-pepsi--1/test/images',
 'train': '/kaggle/working/cola-vs-pepsi--1/train/images',
 'val': 'valid/images',
 'valid': '/kaggle/working/cola-vs-pepsi--1/valid/images'}

## Loading Yolov8

In [14]:
model = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:00<00:00, 182MB/s]


In [15]:

os.environ['WANDB_MODE'] = 'disabled'

## Training the Model

In [16]:
results = model.train(
   data='/kaggle/working/cola-vs-pepsi--1/data.yaml',
   imgsz=640,
   epochs=60,
   batch=8
#    optimizer='Adam',  
#    lr0=0.001         
)


New https://pypi.org/project/ultralytics/8.2.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/cola-vs-pepsi--1/data.yaml, epochs=60, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=Fals

In [17]:
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

## Results

In [18]:
import numpy as np

In [19]:
def detection_timestamps(video_path):
    container = av.open(video_path)
    
    fps = container.streams.video[0].average_rate
    frame_interval = int(fps)  

    detection_dict = {}
    frame_count = 0

    for frame in container.decode(video=0):
        frame_count += 1

        if frame_count % frame_interval == 0:
            current_time = round(frame.time, 2)  

            img = frame.to_ndarray(format='bgr24')
            results = model.predict(img)

            frame_height, frame_width, _ = img.shape
            frame_center = np.array([frame_width / 2, frame_height / 2])

            for result in results:
                if result.boxes is not None:
                    for box in result.boxes:
                        class_id = box.cls if box.cls is not None else -1 
                        if class_id != -1:
                            class_name = result.names[int(class_id)]  
                            
                            xyxy = box.xyxy[0].cpu().numpy()
                            x_min, y_min, x_max, y_max = xyxy

                            width = int(x_max - x_min)
                            height = int(y_max - y_min)
                            size = f"{width}px X {height}px" 

                            x_center = (x_min + x_max) / 2
                            y_center = (y_min + y_max) / 2
                            box_center = np.array([x_center, y_center])
                            distance_from_center = round(np.linalg.norm(box_center - frame_center), 2) 

                            if class_name not in detection_dict:
                                detection_dict[class_name] = []
                            detection_dict[class_name].append({
                                "time_stamp": current_time,
                                "size": size,
                                "distance_from_center": distance_from_center
                            })

    return detection_dict

In [20]:

detections = detection_timestamps('/kaggle/input/video-pepsi-cola/videoplayback.mp4')




0: 384x640 2 Pepsis, 83.3ms
Speed: 2.8ms preprocess, 83.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Pepsis, 7.9ms
Speed: 3.0ms preprocess, 7.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Coca-Cola, 3 Pepsis, 7.8ms
Speed: 2.9ms preprocess, 7.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Pepsis, 7.6ms
Speed: 2.2ms preprocess, 7.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Coca-Cola, 3 Pepsis, 7.6ms
Speed: 3.0ms preprocess, 7.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.6ms
Speed: 2.2ms preprocess, 7.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Pepsis, 7.8ms
Speed: 3.1ms preprocess, 7.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Pepsis, 7.7ms
Speed: 2.2ms preprocess, 7.7ms inference, 2.0ms postprocess per image

In [21]:
import json
# timestamps_json_object = json.dumps(detections,indent = 2)
# print(detections)
timestamps_json_object = json.dumps(detections,indent = 4)

In [22]:
print(timestamps_json_object)

{
    "Pepsi": [
        {
            "time_stamp": 0.93,
            "size": "300px X 495px",
            "distance_from_center": 423.63
        },
        {
            "time_stamp": 0.93,
            "size": "302px X 503px",
            "distance_from_center": 158.97
        },
        {
            "time_stamp": 1.9,
            "size": "302px X 503px",
            "distance_from_center": 425.68
        },
        {
            "time_stamp": 1.9,
            "size": "306px X 507px",
            "distance_from_center": 161.38
        },
        {
            "time_stamp": 2.87,
            "size": "294px X 425px",
            "distance_from_center": 439.9
        },
        {
            "time_stamp": 2.87,
            "size": "308px X 456px",
            "distance_from_center": 180.39
        },
        {
            "time_stamp": 2.87,
            "size": "317px X 731px",
            "distance_from_center": 137.09
        },
        {
            "time_stamp": 3.84,
            "

In [23]:
# print(timestamps_json_object)

In [24]:
# CLASS_IDS = {
#     "CocaCola": 0,
#     "Pepsi": 1
# }

In [25]:
# video_path = "/kaggle/input/video-pepsi-cola/videoplayback.mp4"

In [26]:
# def count_frames(video_path):
#     container = av.open(video_path)
#     total_frames = 0
    
#     for frame in container.decode(video=0):
#         total_frames += 1
    
#     return total_frames

In [27]:
# container = av.open(video_path)
# total_frames = count_frames(video_path)
# total_frames

In [28]:
# container = av.open(video_path)
# total_frames = sum(1 for _ in container.decode(video=0))
# print(f"Total number of frames in video: {total_frames}")

In [29]:
# import cv2
# import numpy as np

In [30]:
# detection_results, total_frames = process_video(video_path, frame_skip=5)

In [31]:
# detection_results

In [32]:
# total_frames

In [33]:
# img = cv2.imread('/kaggle/input/colaco/cola.png')
# img_cv2 = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        
#         # Perform object detection on the frame
# results = model(img_cv2)

In [34]:
# import matplotlib.pyplot as plt

In [35]:
# det = model.predict('/kaggle/input/colaco/cola.png')

In [36]:
# det[0].boxes.id

In [37]:
# detections